In [1]:
import pandas as pd
import numpy as np
from google.colab.data_table import DataTable
from google.colab import drive
from google.colab import files

In [2]:
drive.mount('/content/drive')
pd.set_option('display.max_columns', None)
DataTable.max_columns = 99

Mounted at /content/drive


# **1.Data Preparation**

In [3]:
path = '/content/drive/Shareddrives/Progetto Big Data/DataSet/labouref-france-departement-quarter-jobseeker.csv'
df = pd.read_csv(path, sep = ';')

path_pop = '/content/drive/Shareddrives/Progetto Big Data/DataSet/demographyref-france-pop-legale-departement.csv'
df_pop = pd.read_csv(path_pop, sep = ';')

## 1.1.Comprensione del dataSet e sistemazione delle colonne

In [4]:
df.head(5)

,Période (Trimestre),Date,Catégorie,Tranche d'âge,Nb moyen demandeur emploi,Nb moyen demandeur emploi Homme,Nb moyen demandeur emploi Femme,Différence Nb demandeur emploi trimestre précédent,Nb moyen entrées,Nb moyen sorties,Nb moyen demandeur emploi longue durée,Code Officiel Département,Code Officiel Région,Nom Officiel Région,Code Iso 3166-3 Zone,Nom Officiel Département
0,2012-T2,2012-04,B,Indifférent,8560,NaN,NaN,210.0,NaN,NaN,NaN,77,11,Île-de-France,FXX,Seine-et-Marne
1,2014-T2,2014-04,B,Indifférent,9810,NaN,NaN,10.0,NaN,NaN,NaN,77,11,Île-de-France,FXX,Seine-et-Marne
2,2017-T1,2017-01,B,Indifférent,10960,NaN,NaN,-80.0,NaN,NaN,NaN,77,11,Île-de-France,FXX,Seine-et-Marne
3,2017-T3,2017-07,B,Indifférent,11440,NaN,NaN,420.0,NaN,NaN,NaN,77,11,Île-de-France,FXX,Seine-et-Marne
4,2009-T2,2009-04,B,Indifférent,7050,NaN,NaN,340.0,NaN,NaN,NaN,78,11,Île-de-France,FXX,Yvelines


In [5]:
df_pop.head(5)

,Code Officiel Région,Nom Officiel Région,Code Officiel Département,Nom Officiel Département,Nombre d’arrondissements départementaux,Nombre de cantons,Population municipale Département,Population totale Département,Année de recensement,Année d’entrée en vigueur,Année de référence géographique
0,53,Bretagne,35,Ille-et-Vilaine,4.0,27.0,1079498,1104397,2019,2022,2021
1,11,Île-de-France,91,Essonne,3.0,21.0,1301659,1316011,2019,2022,2021
2,11,Île-de-France,94,Val-de-Marne,3.0,25.0,1407124,1415845,2019,2022,2021
3,76,Occitanie,34,Hérault,3.0,25.0,1175623,1193790,2019,2022,2021
4,84,Auvergne-Rhône-Alpes,38,Isère,3.0,29.0,1271166,1296116,2019,2022,2021


In [6]:
#Rimuovo colonne con valori null
columns = ['Nb moyen demandeur emploi Homme',	'Nb moyen demandeur emploi Femme','Nb moyen entrées','Nb moyen sorties','Nb moyen demandeur emploi longue durée']
df = df.drop(columns, axis=1)

columns_pop = ['Année de recensement',	'Année d’entrée en vigueur','Année de référence géographique']
df_pop= df_pop.drop(columns_pop, axis=1)


In [7]:
#Prendo in considerazione solo tranche d age = indifferent
#df = df[df["Tranche d'âge"] == "Indifférent"]

In [8]:
#Associo la popolazione al dataset
df['Population Department'] = df['Nom Officiel Département'].map(df_pop.set_index('Nom Officiel Département')['Population totale Département'])

In [9]:
#Calcolo percentuale disoccupazione
percentuale = (100*df['Nb moyen demandeur emploi'])/df['Population Department']
df['Pourcentage de chômage'] = percentuale

In [10]:
#Separazione del periodo e del trimestre
df_separated = df['Période (Trimestre)'].str.split('-', expand=True)
df_separated.columns = ['Période', 'Trimestre']
df_separated.head()

,Période,Trimestre
0,2012,T2
1,2014,T2
2,2017,T1
3,2017,T3
4,2009,T2


In [11]:
df = pd.concat([df, df_separated], axis=1)
df = df.drop('Période (Trimestre)',axis = 1)
df.head()

,Date,Catégorie,Tranche d'âge,Nb moyen demandeur emploi,Différence Nb demandeur emploi trimestre précédent,Code Officiel Département,Code Officiel Région,Nom Officiel Région,Code Iso 3166-3 Zone,Nom Officiel Département,Population Department,Pourcentage de chômage,Période,Trimestre
0,2012-04,B,Indifférent,8560,210.0,77,11,Île-de-France,FXX,Seine-et-Marne,1437729,0.595383,2012,T2
1,2014-04,B,Indifférent,9810,10.0,77,11,Île-de-France,FXX,Seine-et-Marne,1437729,0.682326,2014,T2
2,2017-01,B,Indifférent,10960,-80.0,77,11,Île-de-France,FXX,Seine-et-Marne,1437729,0.762313,2017,T1
3,2017-07,B,Indifférent,11440,420.0,77,11,Île-de-France,FXX,Seine-et-Marne,1437729,0.795699,2017,T3
4,2009-04,B,Indifférent,7050,340.0,78,11,Île-de-France,FXX,Yvelines,1473221,0.478543,2009,T2


In [12]:
df.describe(include = 'all')

,Date,Catégorie,Tranche d'âge,Nb moyen demandeur emploi,Différence Nb demandeur emploi trimestre précédent,Code Officiel Département,Code Officiel Région,Nom Officiel Région,Code Iso 3166-3 Zone,Nom Officiel Département,Population Department,Pourcentage de chômage,Période,Trimestre
count,97200,97200,97200,97200.000000,96300.000000,97200,97200.000000,97200,97200,97200,9.720000e+04,97200.000000,97200,97200
unique,108,6,4,NaN,NaN,100,NaN,17,5,100,NaN,NaN,27,4
top,2012-04,ABC,Indifférent,NaN,NaN,77,NaN,Occitanie,FXX,Seine-et-Marne,NaN,NaN,2012,T2
freq,900,43200,64800,NaN,NaN,972,NaN,12636,93312,972,NaN,NaN,3600,24300
mean,NaN,NaN,NaN,15903.002675,48.543510,NaN,53.120000,NaN,NaN,NaN,6.822920e+05,2.280139,NaN,NaN
std,NaN,NaN,NaN,24571.402102,735.016913,NaN,27.945548,NaN,NaN,NaN,5.163869e+05,2.360415,NaN,NaN
min,NaN,NaN,NaN,20.000000,-16020.000000,NaN,1.000000,NaN,NaN,NaN,8.030400e+04,0.007040,NaN,NaN
25%,NaN,NaN,NaN,2660.000000,-80.000000,NaN,27.000000,NaN,NaN,NaN,3.046292e+05,0.614417,NaN,NaN
50%,NaN,NaN,NaN,6820.000000,30.000000,NaN,52.000000,NaN,NaN,NaN,5.447785e+05,1.121382,NaN,NaN
75%,NaN,NaN,NaN,17800.000000,160.000000,NaN,76.000000,NaN,NaN,NaN,8.534378e+05,3.736974,NaN,NaN


## 1.2.Gestione dei valori nulli

In [13]:
missing_data = df.isnull()
missing_data.head()

,Date,Catégorie,Tranche d'âge,Nb moyen demandeur emploi,Différence Nb demandeur emploi trimestre précédent,Code Officiel Département,Code Officiel Région,Nom Officiel Région,Code Iso 3166-3 Zone,Nom Officiel Département,Population Department,Pourcentage de chômage,Période,Trimestre
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [14]:
for column in missing_data.columns.values.tolist():
    #print(column)
    print (missing_data[column].value_counts())
    print("")

False    97200
Name: Date, dtype: int64

False    97200
Name: Catégorie, dtype: int64

False    97200
Name: Tranche d'âge, dtype: int64

False    97200
Name: Nb moyen demandeur emploi, dtype: int64

False    96300
True       900
Name: Différence Nb demandeur emploi trimestre précédent, dtype: int64

False    97200
Name: Code Officiel Département, dtype: int64

False    97200
Name: Code Officiel Région, dtype: int64

False    97200
Name: Nom Officiel Région, dtype: int64

False    97200
Name: Code Iso 3166-3 Zone, dtype: int64

False    97200
Name: Nom Officiel Département, dtype: int64

False    97200
Name: Population Department, dtype: int64

False    97200
Name: Pourcentage de chômage, dtype: int64

False    97200
Name: Période, dtype: int64

False    97200
Name: Trimestre, dtype: int64



## 1.3.Gestione dei tipi

In [15]:
df.dtypes

Date                                                   object
Catégorie                                              object
Tranche d'âge                                          object
Nb moyen demandeur emploi                               int64
Différence Nb demandeur emploi trimestre précédent    float64
Code Officiel Département                              object
Code Officiel Région                                    int64
Nom Officiel Région                                    object
Code Iso 3166-3 Zone                                   object
Nom Officiel Département                               object
Population Department                                   int64
Pourcentage de chômage                                float64
Période                                                object
Trimestre                                              object
dtype: object

In [16]:
df[['Trimestre','Catégorie','Tranche d\'âge','Nom Officiel Région','Code Iso 3166-3 Zone','Nom Officiel Département','Code Officiel Département']] = df[['Trimestre','Catégorie','Tranche d\'âge','Nom Officiel Région','Code Iso 3166-3 Zone','Nom Officiel Département','Code Officiel Département']].astype("string")
df['Période'] = df['Période'].astype('int64')
df['Date'] = pd.to_datetime(df['Date'])

## 1.4 Eliminazione dei dipartimenti del gruppo 1




In [17]:
df.shape

(97200, 14)

In [18]:
df = df.drop(df[df['Nom Officiel Région'] == 'Hauts-de-France'].index)
df = df.drop(df[df['Nom Officiel Région'] == 'Grand Est'].index)
df = df.drop(df[df['Nom Officiel Région'] == 'Île-de-France'].index)
df = df.drop(df[df['Nom Officiel Région'] == 'Normandie'].index)
df = df.drop(df[df['Nom Officiel Région'] == 'Bretagne'].index)
df = df.drop(df[df['Nom Officiel Région'] == 'Bourgogne-Franche-Comté'].index)
df = df.drop(df[df['Nom Officiel Région'] == 'Centre-Val de Loire'].index)

In [19]:
lista_nome_regioni = df['Nom Officiel Région'].unique()
print(lista_nome_regioni)
print(df.shape)

<StringArray>
[                 'Occitanie',           'Pays de la Loire',
 "Provence-Alpes-Côte d'Azur",                 'Guadeloupe',
                 'Martinique',                     'Guyane',
                 'La Réunion',         'Nouvelle-Aquitaine',
       'Auvergne-Rhône-Alpes',                      'Corse']
Length: 10, dtype: string
(52488, 14)


In [20]:
lista_dipartimenti = df['Nom Officiel Département'].unique()
print(lista_dipartimenti)
lista_dipartimenti.size

<StringArray>
[                   'Aude',                    'Gard',
                 'Hérault',                  'Lozère',
     'Pyrénées-Orientales',                  'Ariège',
                 'Aveyron',           'Haute-Garonne',
                    'Gers',                     'Lot',
         'Hautes-Pyrénées',                    'Tarn',
         'Tarn-et-Garonne',        'Loire-Atlantique',
          'Maine-et-Loire',                 'Mayenne',
                  'Sarthe',                  'Vendée',
 'Alpes-de-Haute-Provence',            'Hautes-Alpes',
         'Alpes-Maritimes',        'Bouches-du-Rhône',
                     'Var',                'Vaucluse',
              'Guadeloupe',              'Martinique',
                  'Guyane',              'La Réunion',
                'Dordogne',                 'Gironde',
                  'Landes',          'Lot-et-Garonne',
    'Pyrénées-Atlantiques',                 'Corrèze',
                  'Creuse',            'Haute-Vienn

54

In [21]:
duplicati_presenti = df.duplicated().any()

if duplicati_presenti:
    print("Sono presenti duplicati nel DataFrame.")
else:
    print("Non ci sono duplicati nel DataFrame.")

Non ci sono duplicati nel DataFrame.


## 1.5.Salvataggio del dataSet pulito

In [22]:
path = '/content/drive/Shareddrives/Progetto Big Data/DataSet Puliti/labouref-france-departement-quarter-jobseeker_cleaned.csv'
df.to_csv(path,index=False , sep = ';');